<a href="https://colab.research.google.com/github/anon12a/ed-vae/blob/main/Main_Experiment_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import pandas as pd

from keras.layers import *
from keras.layers import Layer, Dense, Activation, Flatten, Lambda, Conv2D, Conv2DTranspose, BatchNormalization, Reshape
from keras.models import Model, Input, Sequential
from keras.callbacks import EarlyStopping
from keras.losses import mse
from keras import backend as K
from keras import optimizers, initializers, regularizers, constraints
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, KernelPCA
from keras.datasets import mnist, fashion_mnist

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
%matplotlib inline 

#Utilities

In [ ]:
def flatten_data(data):

  data = np.reshape(data,(-1, original_dim))
  
  return data

#Losses and Metrics

In [ ]:
def reconstruction_error(model):

    recon_error = mse(K.reshape(model.input, [-1]), K.reshape(model.output, [-1]))  
    recon_error *= original_dim
    recon_error = K.mean(recon_error)

    return recon_error     

In [ ]:
def kl_divergence(encoder):

    kl = K.mean(- 0.5 * K.sum(1 + encoder.get_output_at(1)[1] - K.square(encoder.get_output_at(1)[0]) - K.exp(encoder.get_output_at(1)[1]), axis=-1))

    return kl

In [ ]:
def per_example_reconstruction_error(model, data):

    decoded = model.predict(data, batch_size=batch_size)
    recon_error = mse(np.reshape(data,(-1,original_dim)), np.reshape(decoded,(-1,original_dim)))
    recon_error *= original_dim

    return tf.Session().run(recon_error)

In [ ]:
# reconstruction error of the expected latent representation 
def per_example_elr_reconstruction_error(encoder, decoder, data):

    z_mean, z_log_var, z = encoder.predict(data, batch_size=batch_size)
    decoded = decoder.predict(z_mean, batch_size=batch_size)
    recon_error = mse(np.reshape(data,(-1,original_dim)), np.reshape(decoded,(-1,original_dim)))
    recon_error *= original_dim

    return tf.Session().run(recon_error)

In [ ]:
def per_example_kl_divergence(encoder, data):

    z_mean, z_log_var, z = encoder.predict(data, batch_size=batch_size)

    kl = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl = K.sum(kl, axis=-1)
    kl *= -0.5

    return tf.Session().run(kl)

In [ ]:
def pr_auc(x, y):
    precision, recall, _ = precision_recall_curve(x, y, pos_label=1)
    pr_auc = auc(recall, precision)
    
    return pr_auc    

In [ ]:
def roc_auc(x, y):
    fpr, tpr, thresholds = roc_curve(x, y, pos_label=1)
    roc_auc = auc(fpr, tpr)

    return roc_auc 

# Networks

## Convolutional Autoencoder

In [ ]:
def conv_encoder(input_side=32, n_channels=3, representation_dim=512, filter_size = 64, representation_activation='sigmoid', intermediate_activation='relu', batch_norm = False):
  
    input_shape = (input_side, input_side, n_channels)

    input = Input(shape=input_shape)

    x = Conv2D(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(input)
    if batch_norm == True:
      x = BatchNormalization()(x)
    x = Activation(intermediate_activation)(x)

    x = Conv2D(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    if batch_norm == True:
      x = BatchNormalization()(x) 
    x = Activation(intermediate_activation)(x)

    x = Flatten()(x)

    x = Dense(filter_size * input_side//4 * input_side//4)(x)
    if batch_norm == True:
      x = BatchNormalization()(x) 
    x = Activation(intermediate_activation)(x)

    x = Dense(representation_dim)(x)
    if batch_norm == True:
      x = BatchNormalization()(x)
    encoded = Activation(representation_activation)(x)   

    return Model(input, encoded, name="encoder")

In [ ]:
def conv_decoder(output_side=32, n_channels=3, representation_dim=512, filter_size = 64, activation='relu', batch_norm = False):

    encoded_input = Input(shape=(representation_dim,))

    x = Dense(filter_size * output_side//4 * output_side//4)(encoded_input)
    if batch_norm == True:
      x = BatchNormalization()(x) 
    x = Activation(activation)(x)

    conv_shape = (output_side//4, output_side//4, filter_size)
    x = Reshape(conv_shape)(x)
    
    x = Conv2DTranspose(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    if batch_norm == True:
      x = BatchNormalization()(x)    
    x = Activation(activation)(x)

    x = Conv2DTranspose(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    if batch_norm == True:
      x = BatchNormalization()(x)     
    x = Activation(activation)(x)

    x = Conv2DTranspose(n_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    if batch_norm == True:
      x = BatchNormalization()(x)    
    decoded = Activation('sigmoid')(x)

    return Model(encoded_input, decoded, name="decoder")

In [ ]:
def convolutional_autoencoder(encoder, decoder):

    input = Input(batch_shape=encoder.input_shape)
    output = decoder(encoder(input))

    cae = Model(input, output, name='cae')

    return cae

## Convolutional Variational Autoencoder

In [ ]:
def cvae_encoder(input_side=32, n_channels=3, representation_dim=512, filter_size = 64, intermediate_activation='relu', batch_norm=False):  
    input_shape = (input_side, input_side, n_channels)

    input = Input(shape=input_shape)

    x = Conv2D(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(input)
    x = Activation(intermediate_activation)(x)

    x = Conv2D(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Activation(intermediate_activation)(x)

    x = Flatten()(x)

    x = Dense(filter_size * input_side//4 * input_side//4)(x)
    x = Activation(intermediate_activation)(x)

    z_mean = Dense(representation_dim, name='z_mean')(x)
    z_log_var = Dense(representation_dim, name='z_log_var')(x)
    z = Lambda(sampling, output_shape=(representation_dim,), name='z')([z_mean, z_log_var])

    return Model(input, [z_mean, z_log_var, z], name="encoder")

In [ ]:
def cvae_decoder(output_side=32, n_channels=3, representation_dim=512, filter_size = 64, activation='relu', batch_norm=False):
    
    encoded_input = Input(shape=(representation_dim,))

    x = Dense(filter_size * output_side//4 * output_side//4)(encoded_input)
    x = Activation(activation)(x)

    conv_shape = (output_side//4, output_side//4, filter_size)
    x = Reshape(conv_shape)(x)

    x = Conv2DTranspose(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Activation(activation)(x)

    x = Conv2DTranspose(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Activation(activation)(x)

    x = Conv2DTranspose(n_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    decoded = Activation('sigmoid')(x)

    return Model(encoded_input, decoded, name="decoder")

In [ ]:
def convolutional_variational_autoencoder(encoder, decoder):

    input = Input(batch_shape=encoder.input_shape)
    output = decoder(encoder(input)[2])

    cvae = Model(input, output, name='cvae')

    return cvae

In [ ]:
# Reparamaterization Trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

#Comparable Models

## Convolutional Autoencoder

In [ ]:
def cae_1(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, input_side, n_channels, latent_dim, filter_size, batch_norm = False):

  encoder = conv_encoder(input_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size = filter_size, batch_norm = batch_norm)
  decoder = conv_decoder(output_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size = filter_size, batch_norm = batch_norm)
  cae = convolutional_autoencoder(encoder, decoder) 

  #encoder.summary()
  #decoder.summary() 

  recon_error = reconstruction_error(cae)

  cae.add_loss(recon_error)
  adam = optimizers.Adam(lr=learning_rate)
  cae.compile(optimizer=adam)

  es = EarlyStopping(monitor='val_loss', mode='min', patience = 20, verbose=1)

  history = cae.fit(x_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(x_val, None),
                  callbacks=[es],
                  verbose=0)
  
  rec_results = per_example_reconstruction_error(cae, x_test)

  pr_auc_rec_result = pr_auc(y_test, rec_results)
  roc_auc_rec_result = roc_auc(y_test, rec_results)

  return pr_auc_rec_result, roc_auc_rec_result, pr_auc_rec_result, roc_auc_rec_result, pr_auc_rec_result, roc_auc_rec_result

## PCA (via SVD)

In [ ]:
def pca_1(x_train, y_train, x_val, x_test, y_test, input_side, n_channels, latent_dim):

    pca = PCA(n_components=latent_dim, svd_solver='full')

    pca.fit(flatten_data(x_train))
    encoded = pca.transform(flatten_data(x_test))
    recon = pca.inverse_transform(encoded)

    recon_error = mse(flatten_data(x_test), recon)
    recon_error *= original_dim
    recon_error = tf.Session().run(recon_error)

    pr_auc_result = pr_auc(y_test, recon_error)
    roc_auc_result = roc_auc(y_test, recon_error)

    return pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result

## KernalPCA

In [ ]:
def kpca_1(x_train, y_train, x_val, x_test, y_test, input_side, n_channels, latent_dim):

    kpca = KernelPCA(n_components=latent_dim, kernel='rbf', fit_inverse_transform="True")

    kpca.fit(flatten_data(x_train))
    encoded = kpca.transform(flatten_data(x_test))
    recon = kpca.inverse_transform(encoded)

    recon_error = mse(flatten_data(x_test), recon)
    recon_error *= original_dim
    recon_error = tf.Session().run(recon_error)

    pr_auc_result = pr_auc(y_test, recon_error)
    roc_auc_result = roc_auc(y_test, recon_error)

    return pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result

## Deep Structured Energy Based Anomaly Detection (DSEBM)

In [ ]:
# From the following implementation: https://github.com/izikgo/AnomalyDetectionTransformations/blob/master/models/dsebm.py
# Licensed under the MIT License
class Prior(Layer):
    def __init__(self,
                 bias_initializer='zeros',
                 bias_regularizer=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(Prior, self).__init__(**kwargs)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.bias = None

    def build(self, input_shape):
        self.bias = self.add_weight(shape=input_shape[1:],
                                    initializer=self.bias_initializer,
                                    name='bias',
                                    regularizer=self.bias_regularizer,
                                    constraint=self.bias_constraint)
        super(Prior, self).build(input_shape)

    def compute_output_shape(self, input_shape):
        return input_shape[0], 1

    def call(self, x, **kwargs):
        return K.sum(K.batch_flatten(K.square(K.bias_add(x, -self.bias))), axis=-1, keepdims=True)


def create_energy_model(encoder_mdl):
    x_in = Input(batch_shape=encoder_mdl.input_shape)

    encoded = encoder_mdl(x_in)
    prior = Prior()(x_in)
    energy = Lambda(lambda args: 0.5*args[0] - K.sum(K.batch_flatten(args[1]), axis=-1, keepdims=True),
                    output_shape=(1,))([prior, encoded])
    return Model(x_in, energy)


def create_reconstruction_model(energy_mdl):
    x_in = Input(batch_shape=energy_mdl.input_shape)
    x = GaussianNoise(stddev=.5)(x_in)  # only active in training
    energy = energy_mdl(x)
    rec = Lambda(lambda args: args[1] - K.gradients(args[0], args[1]),
                 output_shape=energy_mdl.input_shape[1:])([energy, x])

    return Model(x_in, rec)

In [ ]:
def dsebm_1(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, input_side, n_channels, latent_dim, filter_size):
  
  encoder_mdl = conv_encoder(input_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size=filter_size)
  energy_mdl = create_energy_model(encoder_mdl)
  reconstruction_mdl = create_reconstruction_model(energy_mdl)

  adam = optimizers.Adam(lr=learning_rate)
  reconstruction_mdl.compile(optimizer=adam, loss="mse")

  es = EarlyStopping(monitor='val_loss', mode='min', patience = 20, verbose=1)

  history = reconstruction_mdl.fit(x=x_train, y=x_train, 
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(x_val, x_val),
                  callbacks=[es],
                  verbose=0)
  
  scores = -energy_mdl.predict(x_test, batch_size)

  pr_auc_result = pr_auc(y_test, -scores)
  roc_auc_result = roc_auc(y_test, -scores)

  return pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result

# ED-VAE and VAE

In [ ]:
def edvae_vae_comparison(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, input_side, n_channels, latent_dim, filter_size, batch_norm=False, beta = 1.0):

  encoder = cvae_encoder(input_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size = filter_size, batch_norm = batch_norm)
  decoder = cvae_decoder(output_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size = filter_size, batch_norm = batch_norm)
  cae = convolutional_variational_autoencoder(encoder, decoder) 

  #encoder.summary()
  #decoder.summary() 

  recon_error = reconstruction_error(cae)
  kl = kl_divergence(encoder)
  loss = beta * kl + recon_error

  cae.add_loss(loss)
  
  adam = optimizers.Adam(lr=learning_rate)
  cae.compile(optimizer=adam)

  es = EarlyStopping(monitor='val_loss', mode='min', patience = 20, verbose=1)

  history = cae.fit(x_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(x_val, None),
                  callbacks=[es],
                  verbose=0)
  
  # vae results 
  kl_results = per_example_kl_divergence(encoder, x_test)
  rec_results = per_example_reconstruction_error(cae, x_test)
  elbo_results = beta * kl_results + rec_results

  pr_auc_rec_result = pr_auc(y_test, rec_results)
  roc_auc_rec_result = roc_auc(y_test, rec_results)

  pr_auc_elbo_result = pr_auc(y_test, elbo_results)
  roc_auc_elbo_result = roc_auc(y_test, elbo_results)

  pr_auc_kl_result = pr_auc(y_test, kl_results)
  roc_auc_kl_result = roc_auc(y_test, kl_results) 
    
  # ed-vae results
  kl_results = per_example_kl_divergence(encoder, x_test)
  rec_results = per_example_elr_reconstruction_error(encoder, decoder, x_test)
  elbo_results = beta * kl_results + rec_results

  elr_pr_auc_rec_result = pr_auc(y_test, rec_results)
  elr_roc_auc_rec_result = roc_auc(y_test, rec_results)

  elr_pr_auc_elbo_result = pr_auc(y_test, elbo_results)
  elr_roc_auc_elbo_result = roc_auc(y_test, elbo_results)

  elr_pr_auc_kl_result = pr_auc(y_test, kl_results )
  elr_roc_auc_kl_result = roc_auc(y_test, kl_results )

  return np.array([[pr_auc_rec_result, roc_auc_rec_result, pr_auc_elbo_result, roc_auc_elbo_result, pr_auc_kl_result, roc_auc_kl_result],
                   [elr_pr_auc_rec_result, elr_roc_auc_rec_result, elr_pr_auc_elbo_result, elr_roc_auc_elbo_result, elr_pr_auc_kl_result, elr_roc_auc_kl_result]])

# Data Preperation

In [ ]:
def load_mnist():

  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
  x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

  #print('x_train shape:', x_train.shape)
  #print(x_train.shape[0], 'train samples')
  #print(x_test.shape[0], 'test samples')

  return x_train, y_train, x_test, y_test

In [ ]:
def load_fashion_mnist():

  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
  x_train =  x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
  x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

  #print('x_train shape:', x_train.shape)
  #print(x_train.shape[0], 'train samples')
  #print(x_test.shape[0], 'test samples')

  return x_train, y_train, x_test, y_test

In [ ]:
# create training set without anomalies and test set with a ratio p of anomalies. E.g. p = 0.2 
# creates a test set where 20% of the test set are anomalies
def create_data(X_train, Y_train, X_test, Y_test, normal_class, p = .1):

    X_norm_train_filter = np.where(np.isin(Y_train, normal_class))
    X_norm_test_filter = np.where(np.isin(Y_test, normal_class))
    X_anom_test_filter = np.where(np.isin(Y_test, normal_class, invert = True))

    # a = (# of normal instances)/(1 - ratio of outliers)) * ratio of outliers
    # e.g. if p = .2 20% of the test set is anomalies
    a = math.floor(((len(X_norm_test_filter[0]))/(1-p))*p)

    X_anom_test_filter = (np.random.choice(X_anom_test_filter[0], a, replace=False),)
    X_test_filter = (np.concatenate((X_norm_test_filter[0], X_anom_test_filter[0]), axis=None),)

    X_train, Y_train = X_train[X_norm_train_filter[0]], Y_train[X_norm_train_filter]
    X_test_norm, Y_test_norm = X_test[X_norm_test_filter[0]], Y_test[X_norm_test_filter]
    Y_test_class_labels = Y_test[X_test_filter]
    X_test, Y_test, = X_test[X_test_filter[0]], Y_test[X_test_filter]

    Y_test[np.isin(Y_test_class_labels, normal_class)] = 0
    Y_test[np.isin(Y_test_class_labels, normal_class, invert = True)] = 1
    
    return X_train, Y_train, X_test, Y_test, Y_test_class_labels

# Configuration

In [ ]:
image_shape = (28, 28, 1)
original_dim = image_shape[0] * image_shape[1] * image_shape[2]
input_shape = image_shape
batch_size = 128
latent_dim = 32
beta = 1.0
epochs = 1000
learning_rate = 0.0001
activation = 'relu'
filter_size = 32
p = 0.2

# Experiments

In [ ]:
normal_classes = np.arange(0,10)
num_models = 6
num_measures = 6
reps = 10

In [ ]:
def models(x_train, y_train, x_val, x_test, y_test):

  pca_1_results = pca_1(x_train, y_train, x_val, x_test, y_test, image_shape[0], image_shape[2], latent_dim)
  kpca_1_results = kpca_1(x_train, y_train, x_val, x_test, y_test, image_shape[0], image_shape[2], latent_dim)
  dsebm_1_results = dsebm_1(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, image_shape[0], image_shape[2], latent_dim, filter_size)
  cae_1_results = cae_1(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, image_shape[0], image_shape[2], latent_dim, filter_size, batch_norm=False)
  edvae_vae_comparison_results = edvae_vae_comparison(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, image_shape[0], image_shape[2], latent_dim, filter_size, beta = beta)
  
  return pca_1_results, kpca_1_results, dsebm_1_results, cae_1_results, edvae_vae_comparison_results[0], edvae_vae_comparison_results[1]

In [ ]:
results = np.zeros((reps, len(normal_classes), num_models, num_measures))

from tqdm import tqdm

for rep in tqdm(range(reps), desc="reps"):
  for normal in tqdm(range(len(normal_classes)), desc="anomaly"):
    x_train, y_train, x_test, y_test = load_mnist()
    x_train, y_train, x_test, y_test, y_test_class_labels = create_data(x_train, y_train, x_test, y_test, normal_classes[normal], p = p)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    results[rep, normal] = models(x_train, y_train, x_val, x_test, y_test)
    tf.keras.backend.clear_session() #release gpu memory

anomaly:   0%|          | 0/10 [00:00<?, ?it/s]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 00092: early stopping


/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


Epoch 00201: early stopping


/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


...

/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


Epoch 00161: early stopping


/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


Epoch 00316: early stopping



reps: 100%|██████████| 10/10 [5:11:07<00:00, 1866.76s/it]


# Results - ED-VAE vs. Comparable Models

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
index_labels = normal_classes

In [ ]:
avg_results = np.mean(results, axis=0)
std_results = np.std(results, axis=0)
avg_class_results = np.mean(results, axis=(0, 1))
std_class_results = np.std(results, axis=(0, 1))

In [ ]:
pr_header = [np.array(['pca','pca',
                    'kpca','kpca',
                    'dsebm','dsebm',
                    'ae','ae',
                    'vae', 'vae',             
                    'ed-vae', 'ed-vae',
                    ]), 
np.array(['pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          ])] 

In [ ]:
auc_header = [np.array(['pca','pca',
                    'kpca','kpca',
                    'dsebm','dsebm',
                    'ae','ae',
                    'vae', 'vae',             
                    'ed-vae', 'ed-vae',
                    ]), 
np.array(['auc_mean','auc_std',
          'auc_mean','auc_std',
          'auc_mean','auc_std',
          'auc_mean','auc_std',
          'auc_mean','auc_std',
          'auc_mean','auc_std',
          ])] 

### $ELBO$
Compare ELBO of VAE and ED-VAE to results of competitive models


In [ ]:
# AUC
avg_auc_elbo_class_results = pd.DataFrame((avg_class_results[0,3], std_class_results[0,3], 
                                  avg_class_results[1,3], std_class_results[1,3],
                                  avg_class_results[2,3], std_class_results[2,3],  
                                  avg_class_results[3,3], std_class_results[3,3], 
                                  avg_class_results[4,3], std_class_results[4,3], 
                                  avg_class_results[5,3], std_class_results[5,3], 
                                  ), 
                                  index = auc_header).T
avg_auc_elbo_class_results = avg_auc_elbo_class_results.rename(index={0: "mean"})
avg_auc_elbo_final_results = pd.DataFrame((avg_results[:,0,3], std_results[:,0,3],
                                  avg_results[:,1,3], std_results[:,1,3], 
                                  avg_results[:,2,3], std_results[:,2,3], 
                                  avg_results[:,3,3], std_results[:,3,3], 
                                  avg_results[:,4,3], std_results[:,4,3],  
                                  avg_results[:,5,3], std_results[:,5,3], 
                                  ), 
                                  index = auc_header, columns = index_labels).T
avg_auc_elbo_final_results.append(avg_auc_elbo_class_results)

pca                kpca               dsebm                  ae  \
      auc_mean   auc_std  auc_mean   auc_std  auc_mean   auc_std  auc_mean   
0     0.991902  0.001006  0.989760  0.001497  0.536940  0.246504  0.969052   
1     0.998884  0.000132  0.998158  0.000112  0.982074  0.009387  0.987335   
2     0.924497  0.007101  0.896655  0.009141  0.518324  0.132393  0.780557   
3     0.952098  0.004585  0.938679  0.007595  0.508091  0.038067  0.858054   
4     0.943417  0.004557  0.945677  0.004942  0.692532  0.064927  0.882161   
5     0.974173  0.003210  0.950427  0.004540  0.556980  0.024939  0.725093   
6     0.983505  0.001988  0.973489  0.002965  0.584922  0.131559  0.868427   
7     0.970885  0.003842  0.964386  0.005171  0.757129  0.026946  0.912264   
8     0.854871  0.016547  0.854343  0.016311  0.463918  0.102268  0.779581   
9     0.963962  0.001586  0.954091  0.001989  0.663246  0.054227  0.880227   
mean  0.955819  0.040362  0.946566  0.041466  0.626416  0.182221  0.864275   

                     vae              ed-vae            
       auc_std  auc_mean   auc_std  auc_mean   auc_std  
0     0.003922  0.997490  0.000596  0.998040  0.000399  
1     0.001369  0.999143  0.000108  0.998971  0.000175  
2     0.007687  0.930162  0.013765  0.953000  0.011181  
3     0.009869  0.945802  0.011802  0.957721  0.009410  
4     0.009189  0.949664  0.007648  0.963341  0.009216  
5     0.011491  0.958067  0.004985  0.966591  0.005621  
6     0.006613  0.988640  0.003114  0.993058  0.001855  
7     0.008214  0.965191  0.006841  0.972690  0.004912  
8     0.024401  0.894889  0.013219  0.910885  0.015663  
9     0.006204  0.975738  0.003147  0.982728  0.002057  
mean  0.079840  0.960479  0.031821  0.969702  0.026312

In [ ]:
# PR-AUC
avg_pr_elbo_class_results = pd.DataFrame((avg_class_results[0,2], std_class_results[0,2], 
                                  avg_class_results[1,2], std_class_results[1,2],
                                  avg_class_results[2,2], std_class_results[2,2],  
                                  avg_class_results[3,2], std_class_results[3,2], 
                                  avg_class_results[4,2], std_class_results[4,2], 
                                  avg_class_results[5,2], std_class_results[5,2], 
                                  ), 
                                  index = pr_header).T
avg_pr_elbo_class_results = avg_pr_elbo_class_results.rename(index={0: "mean"})
avg_pr_elbo_final_results = pd.DataFrame((avg_results[:,0,2], std_results[:,0,2],
                                  avg_results[:,1,2], std_results[:,1,2], 
                                  avg_results[:,2,2], std_results[:,2,2], 
                                  avg_results[:,3,2], std_results[:,3,2], 
                                  avg_results[:,4,2], std_results[:,4,2],  
                                  avg_results[:,5,2], std_results[:,5,2], 
                                  ), 
                                  index = pr_header, columns = index_labels).T
avg_pr_elbo_final_results.append(avg_pr_elbo_class_results)

pca                   kpca                  dsebm             \
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std   
0       0.962871   0.003811    0.956042   0.005334    0.319372   0.287440   
1       0.995024   0.000561    0.991530   0.000569    0.935702   0.031284   
2       0.808741   0.012070    0.743827   0.016023    0.278496   0.108636   
3       0.808417   0.013630    0.799806   0.016731    0.253902   0.028191   
4       0.876223   0.010232    0.863651   0.010238    0.446265   0.069021   
5       0.907688   0.009187    0.836815   0.011848    0.300565   0.022210   
6       0.948166   0.004434    0.912611   0.006725    0.320903   0.093916   
7       0.913185   0.009281    0.894678   0.011315    0.586697   0.028410   
8       0.698792   0.019722    0.682095   0.018040    0.213635   0.071373   
9       0.879265   0.005546    0.831620   0.008073    0.435122   0.046868   
mean    0.879837   0.084099    0.851267   0.090129    0.409066   0.231569   

              ae                    vae                 ed-vae             
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std  
0       0.884234   0.011367    0.988285   0.003438    0.990907   0.001923  
1       0.954844   0.005402    0.996292   0.000464    0.995649   0.000759  
2       0.498710   0.015766    0.846786   0.021684    0.882667   0.021451  
3       0.613603   0.024578    0.860224   0.023341    0.883761   0.020190  
4       0.666866   0.017487    0.894577   0.013628    0.912480   0.015687  
5       0.443555   0.016242    0.884872   0.011750    0.900353   0.011562  
6       0.546778   0.013287    0.962311   0.008520    0.973199   0.005970  
7       0.751677   0.014373    0.913266   0.011386    0.924547   0.009787  
8       0.460720   0.066113    0.809002   0.019917    0.845131   0.016582  
9       0.622364   0.017670    0.920291   0.008016    0.936706   0.005146  
mean    0.644335   0.166980    0.907590   0.059939    0.924540   0.049106

### $R_{error}$
Compare Reconstruction Error of VAE and ED-VAE to results of competitive models

In [ ]:
# AUC
avg_auc_rec_class_results = pd.DataFrame((avg_class_results[0,1], std_class_results[0,1], 
                                  avg_class_results[1,1], std_class_results[1,1],
                                  avg_class_results[2,1], std_class_results[2,1],  
                                  avg_class_results[3,1], std_class_results[3,1], 
                                  avg_class_results[4,1], std_class_results[4,1], 
                                  avg_class_results[5,1], std_class_results[5,1], 
                                  ), 
                                  index = auc_header).T
avg_auc_rec_class_results = avg_auc_rec_class_results.rename(index={0: "mean"})
avg_auc_rec_final_results = pd.DataFrame((avg_results[:,0,1], std_results[:,0,1], 
                                  avg_results[:,1,1], std_results[:,1,1],
                                  avg_results[:,2,1], std_results[:,2,1],  
                                  avg_results[:,3,1], std_results[:,3,1], 
                                  avg_results[:,4,1], std_results[:,4,1], 
                                  avg_results[:,5,1], std_results[:,5,1], 
                                  ), 
                                  index = auc_header, columns = index_labels).T
avg_auc_rec_final_results.append(avg_auc_rec_class_results)

pca                kpca               dsebm                  ae  \
      auc_mean   auc_std  auc_mean   auc_std  auc_mean   auc_std  auc_mean   
0     0.991902  0.001006  0.989760  0.001497  0.536940  0.246504  0.969052   
1     0.998884  0.000132  0.998158  0.000112  0.982074  0.009387  0.987335   
2     0.924497  0.007101  0.896655  0.009141  0.518324  0.132393  0.780557   
3     0.952098  0.004585  0.938679  0.007595  0.508091  0.038067  0.858054   
4     0.943417  0.004557  0.945677  0.004942  0.692532  0.064927  0.882161   
5     0.974173  0.003210  0.950427  0.004540  0.556980  0.024939  0.725093   
6     0.983505  0.001988  0.973489  0.002965  0.584922  0.131559  0.868427   
7     0.970885  0.003842  0.964386  0.005171  0.757129  0.026946  0.912264   
8     0.854871  0.016547  0.854343  0.016311  0.463918  0.102268  0.779581   
9     0.963962  0.001586  0.954091  0.001989  0.663246  0.054227  0.880227   
mean  0.955819  0.040362  0.946566  0.041466  0.626416  0.182221  0.864275   

                     vae              ed-vae            
       auc_std  auc_mean   auc_std  auc_mean   auc_std  
0     0.003922  0.997371  0.000639  0.997917  0.000565  
1     0.001369  0.999397  0.000117  0.999129  0.000161  
2     0.007687  0.927271  0.013782  0.956265  0.009806  
3     0.009869  0.941618  0.010844  0.956757  0.008551  
4     0.009189  0.946698  0.010322  0.963841  0.011562  
5     0.011491  0.958839  0.004844  0.968716  0.005531  
6     0.006613  0.990615  0.002973  0.994452  0.001617  
7     0.008214  0.960097  0.007662  0.969931  0.005128  
8     0.024401  0.895123  0.013470  0.910591  0.017666  
9     0.006204  0.978472  0.003114  0.985888  0.001568  
mean  0.079840  0.959550  0.032739  0.970349  0.026622

In [ ]:
# PR-AUC
avg_pr_rec_class_results = pd.DataFrame((avg_class_results[0,0], std_class_results[0,0], 
                                  avg_class_results[1,0], std_class_results[1,0],
                                  avg_class_results[2,0], std_class_results[2,0],  
                                  avg_class_results[3,0], std_class_results[3,0], 
                                  avg_class_results[4,0], std_class_results[4,0], 
                                  avg_class_results[5,0], std_class_results[5,0], 
                                  ), 
                                  index = pr_header).T
avg_pr_rec_class_results = avg_pr_rec_class_results.rename(index={0: "mean"})
avg_pr_rec_final_results = pd.DataFrame((avg_results[:,0,0], std_results[:,0,0], 
                                  avg_results[:,1,0], std_results[:,1,0],
                                  avg_results[:,2,0], std_results[:,2,0],  
                                  avg_results[:,3,0], std_results[:,3,0], 
                                  avg_results[:,4,0], std_results[:,4,0], 
                                  avg_results[:,5,0], std_results[:,5,0], 
                                  ), 
                                  index = pr_header, columns = index_labels).T
avg_pr_rec_final_results.append(avg_pr_rec_class_results)

pca                   kpca                  dsebm             \
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std   
0       0.962871   0.003811    0.956042   0.005334    0.319372   0.287440   
1       0.995024   0.000561    0.991530   0.000569    0.935702   0.031284   
2       0.808741   0.012070    0.743827   0.016023    0.278496   0.108636   
3       0.808417   0.013630    0.799806   0.016731    0.253902   0.028191   
4       0.876223   0.010232    0.863651   0.010238    0.446265   0.069021   
5       0.907688   0.009187    0.836815   0.011848    0.300565   0.022210   
6       0.948166   0.004434    0.912611   0.006725    0.320903   0.093916   
7       0.913185   0.009281    0.894678   0.011315    0.586697   0.028410   
8       0.698792   0.019722    0.682095   0.018040    0.213635   0.071373   
9       0.879265   0.005546    0.831620   0.008073    0.435122   0.046868   
mean    0.879837   0.084099    0.851267   0.090129    0.409066   0.231569   

              ae                    vae                 ed-vae             
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std  
0       0.884234   0.011367    0.988409   0.003320    0.990896   0.002023  
1       0.954844   0.005402    0.997493   0.000477    0.996406   0.000687  
2       0.498710   0.015766    0.842210   0.022451    0.886398   0.020869  
3       0.613603   0.024578    0.857914   0.020537    0.884915   0.016641  
4       0.666866   0.017487    0.897972   0.015864    0.917361   0.018964  
5       0.443555   0.016242    0.894223   0.010736    0.910305   0.011131  
6       0.546778   0.013287    0.967355   0.007963    0.977310   0.005809  
7       0.751677   0.014373    0.909373   0.011026    0.921433   0.009237  
8       0.460720   0.066113    0.818364   0.018461    0.855947   0.016364  
9       0.622364   0.017670    0.928812   0.007496    0.945767   0.004407  
mean    0.644335   0.166980    0.910212   0.059524    0.928674   0.047152

## Formated for Publication
Mean and SEM


In [ ]:
# returns the list for df.style.apply() method
def highlight_max(s):
    s = s.str.split(pat='±',expand=True)[0]
    is_max = s == s.max()
    #print(s)
    #print(is_max)
    #return ['background: lightgreen' if cell else '' for cell in is_max]
    return ['font-weight: bold' if cell else '' for cell in is_max]

###ELBO
Compare ELBO of VAE and ED-VAE to results of competitive models

In [ ]:
# AUC
df = pd.DataFrame((
    round(avg_auc_elbo_final_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_elbo_final_results[column,'auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_auc_elbo_final_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_elbo_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_auc_elbo_class_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_elbo_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_auc_elbo_class_results.keys().get_level_values(0)[::2]
    round(avg_auc_elbo_class_results[column,'auc_mean']*100,1).astype(str) for column in avg_auc_elbo_class_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_elbo_class_results.keys().get_level_values(0)[::2]).T)
df

,pca,kpca,dsebm,ae,vae,ed-vae
0,99.2±0.0,99.0±0.0,53.7±7.8,96.9±0.1,99.7±0.0,99.8±0.0
1,99.9±0.0,99.8±0.0,98.2±0.3,98.7±0.0,99.9±0.0,99.9±0.0
2,92.4±0.2,89.7±0.3,51.8±4.2,78.1±0.2,93.0±0.4,95.3±0.4
3,95.2±0.1,93.9±0.2,50.8±1.2,85.8±0.3,94.6±0.4,95.8±0.3
4,94.3±0.1,94.6±0.2,69.3±2.1,88.2±0.3,95.0±0.2,96.3±0.3
5,97.4±0.1,95.0±0.1,55.7±0.8,72.5±0.4,95.8±0.2,96.7±0.2
6,98.4±0.1,97.3±0.1,58.5±4.2,86.8±0.2,98.9±0.1,99.3±0.1
7,97.1±0.1,96.4±0.2,75.7±0.9,91.2±0.3,96.5±0.2,97.3±0.2
8,85.5±0.5,85.4±0.5,46.4±3.2,78.0±0.8,89.5±0.4,91.1±0.5
9,96.4±0.1,95.4±0.1,66.3±1.7,88.0±0.2,97.6±0.1,98.3±0.1


In [ ]:
df.style.apply(highlight_max, axis=1)

,pca,kpca,dsebm,ae,vae,ed-vae
0,99.2±0.0,99.0±0.0,53.7±7.8,96.9±0.1,99.7±0.0,99.8±0.0
1,99.9±0.0,99.8±0.0,98.2±0.3,98.7±0.0,99.9±0.0,99.9±0.0
2,92.4±0.2,89.7±0.3,51.8±4.2,78.1±0.2,93.0±0.4,95.3±0.4
3,95.2±0.1,93.9±0.2,50.8±1.2,85.8±0.3,94.6±0.4,95.8±0.3
4,94.3±0.1,94.6±0.2,69.3±2.1,88.2±0.3,95.0±0.2,96.3±0.3
5,97.4±0.1,95.0±0.1,55.7±0.8,72.5±0.4,95.8±0.2,96.7±0.2
6,98.4±0.1,97.3±0.1,58.5±4.2,86.8±0.2,98.9±0.1,99.3±0.1
7,97.1±0.1,96.4±0.2,75.7±0.9,91.2±0.3,96.5±0.2,97.3±0.2
8,85.5±0.5,85.4±0.5,46.4±3.2,78.0±0.8,89.5±0.4,91.1±0.5
9,96.4±0.1,95.4±0.1,66.3±1.7,88.0±0.2,97.6±0.1,98.3±0.1


In [ ]:
# PR-AUC
df = pd.DataFrame((
    round(avg_pr_elbo_final_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_elbo_final_results[column,'pr_auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_pr_elbo_final_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_elbo_final_results.keys().get_level_values(0)[::2]).T  
df = df.append(pd.DataFrame((
    #round(avg_pr_elbo_class_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_elbo_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_pr_elbo_class_results.keys().get_level_values(0)[::2]
    round(avg_pr_elbo_class_results[column,'pr_auc_mean']*100,1).astype(str) for column in avg_pr_elbo_class_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_elbo_class_results.keys().get_level_values(0)[::2]).T)
df

,pca,kpca,dsebm,ae,vae,ed-vae
0,96.3±0.1,95.6±0.2,31.9±9.1,88.4±0.4,98.8±0.1,99.1±0.1
1,99.5±0.0,99.2±0.0,93.6±1.0,95.5±0.2,99.6±0.0,99.6±0.0
2,80.9±0.4,74.4±0.5,27.8±3.4,49.9±0.5,84.7±0.7,88.3±0.7
3,80.8±0.4,80.0±0.5,25.4±0.9,61.4±0.8,86.0±0.7,88.4±0.6
4,87.6±0.3,86.4±0.3,44.6±2.2,66.7±0.6,89.5±0.4,91.2±0.5
5,90.8±0.3,83.7±0.4,30.1±0.7,44.4±0.5,88.5±0.4,90.0±0.4
6,94.8±0.1,91.3±0.2,32.1±3.0,54.7±0.4,96.2±0.3,97.3±0.2
7,91.3±0.3,89.5±0.4,58.7±0.9,75.2±0.5,91.3±0.4,92.5±0.3
8,69.9±0.6,68.2±0.6,21.4±2.3,46.1±2.1,80.9±0.6,84.5±0.5
9,87.9±0.2,83.2±0.3,43.5±1.5,62.2±0.6,92.0±0.3,93.7±0.2


In [ ]:
df.style.apply(highlight_max, axis=1)

,pca,kpca,dsebm,ae,vae,ed-vae
0,96.3±0.1,95.6±0.2,31.9±9.1,88.4±0.4,98.8±0.1,99.1±0.1
1,99.5±0.0,99.2±0.0,93.6±1.0,95.5±0.2,99.6±0.0,99.6±0.0
2,80.9±0.4,74.4±0.5,27.8±3.4,49.9±0.5,84.7±0.7,88.3±0.7
3,80.8±0.4,80.0±0.5,25.4±0.9,61.4±0.8,86.0±0.7,88.4±0.6
4,87.6±0.3,86.4±0.3,44.6±2.2,66.7±0.6,89.5±0.4,91.2±0.5
5,90.8±0.3,83.7±0.4,30.1±0.7,44.4±0.5,88.5±0.4,90.0±0.4
6,94.8±0.1,91.3±0.2,32.1±3.0,54.7±0.4,96.2±0.3,97.3±0.2
7,91.3±0.3,89.5±0.4,58.7±0.9,75.2±0.5,91.3±0.4,92.5±0.3
8,69.9±0.6,68.2±0.6,21.4±2.3,46.1±2.1,80.9±0.6,84.5±0.5
9,87.9±0.2,83.2±0.3,43.5±1.5,62.2±0.6,92.0±0.3,93.7±0.2


### $R_{error}$
Compare Reconstruction Error of VAE and ED-VAE to results of competitive models

In [ ]:
# AUC
df = pd.DataFrame((
    round(avg_auc_rec_final_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_rec_final_results[column,'auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_auc_rec_final_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_rec_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_auc_rec_class_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_rec_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_auc_rec_class_results.keys().get_level_values(0)[::2]
    round(avg_auc_rec_class_results[column,'auc_mean']*100,1).astype(str) for column in avg_auc_rec_class_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_rec_class_results.keys().get_level_values(0)[::2]).T)
df

,pca,kpca,dsebm,ae,vae,ed-vae
0,99.2±0.0,99.0±0.0,53.7±7.8,96.9±0.1,99.7±0.0,99.8±0.0
1,99.9±0.0,99.8±0.0,98.2±0.3,98.7±0.0,99.9±0.0,99.9±0.0
2,92.4±0.2,89.7±0.3,51.8±4.2,78.1±0.2,92.7±0.4,95.6±0.3
3,95.2±0.1,93.9±0.2,50.8±1.2,85.8±0.3,94.2±0.3,95.7±0.3
4,94.3±0.1,94.6±0.2,69.3±2.1,88.2±0.3,94.7±0.3,96.4±0.4
5,97.4±0.1,95.0±0.1,55.7±0.8,72.5±0.4,95.9±0.2,96.9±0.2
6,98.4±0.1,97.3±0.1,58.5±4.2,86.8±0.2,99.1±0.1,99.4±0.1
7,97.1±0.1,96.4±0.2,75.7±0.9,91.2±0.3,96.0±0.2,97.0±0.2
8,85.5±0.5,85.4±0.5,46.4±3.2,78.0±0.8,89.5±0.4,91.1±0.6
9,96.4±0.1,95.4±0.1,66.3±1.7,88.0±0.2,97.8±0.1,98.6±0.0


In [ ]:
df.style.apply(highlight_max, axis=1)

,pca,kpca,dsebm,ae,vae,ed-vae
0,99.2±0.0,99.0±0.0,53.7±7.8,96.9±0.1,99.7±0.0,99.8±0.0
1,99.9±0.0,99.8±0.0,98.2±0.3,98.7±0.0,99.9±0.0,99.9±0.0
2,92.4±0.2,89.7±0.3,51.8±4.2,78.1±0.2,92.7±0.4,95.6±0.3
3,95.2±0.1,93.9±0.2,50.8±1.2,85.8±0.3,94.2±0.3,95.7±0.3
4,94.3±0.1,94.6±0.2,69.3±2.1,88.2±0.3,94.7±0.3,96.4±0.4
5,97.4±0.1,95.0±0.1,55.7±0.8,72.5±0.4,95.9±0.2,96.9±0.2
6,98.4±0.1,97.3±0.1,58.5±4.2,86.8±0.2,99.1±0.1,99.4±0.1
7,97.1±0.1,96.4±0.2,75.7±0.9,91.2±0.3,96.0±0.2,97.0±0.2
8,85.5±0.5,85.4±0.5,46.4±3.2,78.0±0.8,89.5±0.4,91.1±0.6
9,96.4±0.1,95.4±0.1,66.3±1.7,88.0±0.2,97.8±0.1,98.6±0.0


In [ ]:
# PR-AUC
df = pd.DataFrame((
    round(avg_pr_rec_final_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_rec_final_results[column,'pr_auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_pr_rec_final_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_rec_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_pr_rec_class_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_rec_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_pr_rec_class_results.keys().get_level_values(0)[::2]
    round(avg_pr_rec_class_results[column,'pr_auc_mean']*100,1).astype(str) for column in avg_pr_rec_class_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_rec_class_results.keys().get_level_values(0)[::2]).T)
df

,pca,kpca,dsebm,ae,vae,ed-vae
0,96.3±0.1,95.6±0.2,31.9±9.1,88.4±0.4,98.8±0.1,99.1±0.1
1,99.5±0.0,99.2±0.0,93.6±1.0,95.5±0.2,99.7±0.0,99.6±0.0
2,80.9±0.4,74.4±0.5,27.8±3.4,49.9±0.5,84.2±0.7,88.6±0.7
3,80.8±0.4,80.0±0.5,25.4±0.9,61.4±0.8,85.8±0.6,88.5±0.5
4,87.6±0.3,86.4±0.3,44.6±2.2,66.7±0.6,89.8±0.5,91.7±0.6
5,90.8±0.3,83.7±0.4,30.1±0.7,44.4±0.5,89.4±0.3,91.0±0.4
6,94.8±0.1,91.3±0.2,32.1±3.0,54.7±0.4,96.7±0.3,97.7±0.2
7,91.3±0.3,89.5±0.4,58.7±0.9,75.2±0.5,90.9±0.3,92.1±0.3
8,69.9±0.6,68.2±0.6,21.4±2.3,46.1±2.1,81.8±0.6,85.6±0.5
9,87.9±0.2,83.2±0.3,43.5±1.5,62.2±0.6,92.9±0.2,94.6±0.1


In [ ]:
df.style.apply(highlight_max, axis=1)

,pca,kpca,dsebm,ae,vae,ed-vae
0,96.3±0.1,95.6±0.2,31.9±9.1,88.4±0.4,98.8±0.1,99.1±0.1
1,99.5±0.0,99.2±0.0,93.6±1.0,95.5±0.2,99.7±0.0,99.6±0.0
2,80.9±0.4,74.4±0.5,27.8±3.4,49.9±0.5,84.2±0.7,88.6±0.7
3,80.8±0.4,80.0±0.5,25.4±0.9,61.4±0.8,85.8±0.6,88.5±0.5
4,87.6±0.3,86.4±0.3,44.6±2.2,66.7±0.6,89.8±0.5,91.7±0.6
5,90.8±0.3,83.7±0.4,30.1±0.7,44.4±0.5,89.4±0.3,91.0±0.4
6,94.8±0.1,91.3±0.2,32.1±3.0,54.7±0.4,96.7±0.3,97.7±0.2
7,91.3±0.3,89.5±0.4,58.7±0.9,75.2±0.5,90.9±0.3,92.1±0.3
8,69.9±0.6,68.2±0.6,21.4±2.3,46.1±2.1,81.8±0.6,85.6±0.5
9,87.9±0.2,83.2±0.3,43.5±1.5,62.2±0.6,92.9±0.2,94.6±0.1


# Results - ED-VAE vs. VAE

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
index_labels = normal_classes

In [ ]:
avg_results = np.mean(results, axis=0)
std_results = np.std(results, axis=0)
avg_class_results = np.mean(results, axis=(0, 1))
std_class_results = np.std(results, axis=(0, 1))

In [ ]:
avg_results = avg_results[:,4:6]
std_results = std_results[:,4:6]
avg_class_results = avg_class_results[4:6]
std_class_results = std_class_results[4:6]

In [ ]:
pr_header = [np.array(['vae', 'vae',             
                    'ed-vae', 'ed-vae',
                    ]), 
np.array(['pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          ])] 

In [ ]:
auc_header = [np.array([
                    'vae', 'vae',
                    'ed-vae', 'ed-vae',
                    ]), 
np.array(['auc_mean','auc_std',
          'auc_mean','auc_std',
          ])] 

### $ELBO$
Compare ELBO of VAE and ED-VAE


In [ ]:
# AUC
avg_auc_elbo_class_results = pd.DataFrame((avg_class_results[0,3], std_class_results[0,3], 
                                  avg_class_results[1,3], std_class_results[1,3],
                                  ), 
                                  index = auc_header).T
avg_auc_elbo_class_results = avg_auc_elbo_class_results.rename(index={0: "mean"})
avg_auc_elbo_final_results = pd.DataFrame((avg_results[:,0,3], std_results[:,0,3],
                                  avg_results[:,1,3], std_results[:,1,3], 
                                  ), 
                                  index = auc_header, columns = index_labels).T
avg_auc_elbo_final_results.append(avg_auc_elbo_class_results)

vae              ed-vae          
      auc_mean   auc_std  auc_mean   auc_std
0     0.997490  0.000596  0.998040  0.000399
1     0.999143  0.000108  0.998971  0.000175
2     0.930162  0.013765  0.953000  0.011181
3     0.945802  0.011802  0.957721  0.009410
4     0.949664  0.007648  0.963341  0.009216
5     0.958067  0.004985  0.966591  0.005621
6     0.988640  0.003114  0.993058  0.001855
7     0.965191  0.006841  0.972690  0.004912
8     0.894889  0.013219  0.910885  0.015663
9     0.975738  0.003147  0.982728  0.002057
mean  0.960479  0.031821  0.969702  0.026312

In [ ]:
# PR-AUC
avg_pr_elbo_class_results = pd.DataFrame((avg_class_results[0,2], std_class_results[0,2], 
                                  avg_class_results[1,2], std_class_results[1,2],
                                  ), 
                                  index = pr_header).T
avg_pr_elbo_class_results = avg_pr_elbo_class_results.rename(index={0: "mean"})
avg_pr_elbo_final_results = pd.DataFrame((avg_results[:,0,2], std_results[:,0,2],
                                  avg_results[:,1,2], std_results[:,1,2],
                                  ), 
                                  index = pr_header, columns = index_labels).T
avg_pr_elbo_final_results.append(avg_pr_elbo_class_results)

vae                 ed-vae           
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std
0       0.988285   0.003438    0.990907   0.001923
1       0.996292   0.000464    0.995649   0.000759
2       0.846786   0.021684    0.882667   0.021451
3       0.860224   0.023341    0.883761   0.020190
4       0.894577   0.013628    0.912480   0.015687
5       0.884872   0.011750    0.900353   0.011562
6       0.962311   0.008520    0.973199   0.005970
7       0.913266   0.011386    0.924547   0.009787
8       0.809002   0.019917    0.845131   0.016582
9       0.920291   0.008016    0.936706   0.005146
mean    0.907590   0.059939    0.924540   0.049106

### $R_{error}$
Compare Reconstruction Error of VAE and ED-VAE

In [ ]:
# AUC
avg_auc_rec_class_results = pd.DataFrame((avg_class_results[0,1], std_class_results[0,1], 
                                  avg_class_results[1,1], std_class_results[1,1],
                                  ), 
                                  index = auc_header).T
avg_auc_rec_class_results = avg_auc_rec_class_results.rename(index={0: "mean"})
avg_auc_rec_final_results = pd.DataFrame((avg_results[:,0,1], std_results[:,0,1], 
                                  avg_results[:,1,1], std_results[:,1,1],
                                  ), 
                                  index = auc_header, columns = index_labels).T
avg_auc_rec_final_results.append(avg_auc_rec_class_results)

vae              ed-vae          
      auc_mean   auc_std  auc_mean   auc_std
0     0.997371  0.000639  0.997917  0.000565
1     0.999397  0.000117  0.999129  0.000161
2     0.927271  0.013782  0.956265  0.009806
3     0.941618  0.010844  0.956757  0.008551
4     0.946698  0.010322  0.963841  0.011562
5     0.958839  0.004844  0.968716  0.005531
6     0.990615  0.002973  0.994452  0.001617
7     0.960097  0.007662  0.969931  0.005128
8     0.895123  0.013470  0.910591  0.017666
9     0.978472  0.003114  0.985888  0.001568
mean  0.959550  0.032739  0.970349  0.026622

In [ ]:
# PR-AUC
avg_pr_rec_class_results = pd.DataFrame((avg_class_results[0,0], std_class_results[0,0], 
                                  avg_class_results[1,0], std_class_results[1,0],
                                  ), 
                                  index = pr_header).T
avg_pr_rec_class_results = avg_pr_rec_class_results.rename(index={0: "mean"})
avg_pr_rec_final_results = pd.DataFrame((avg_results[:,0,0], std_results[:,0,0], 
                                  avg_results[:,1,0], std_results[:,1,0],
                                  ), 
                                  index = pr_header, columns = index_labels).T
avg_pr_rec_final_results.append(avg_pr_rec_class_results)

vae                 ed-vae           
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std
0       0.988409   0.003320    0.990896   0.002023
1       0.997493   0.000477    0.996406   0.000687
2       0.842210   0.022451    0.886398   0.020869
3       0.857914   0.020537    0.884915   0.016641
4       0.897972   0.015864    0.917361   0.018964
5       0.894223   0.010736    0.910305   0.011131
6       0.967355   0.007963    0.977310   0.005809
7       0.909373   0.011026    0.921433   0.009237
8       0.818364   0.018461    0.855947   0.016364
9       0.928812   0.007496    0.945767   0.004407
mean    0.910212   0.059524    0.928674   0.047152

## Formated for Publication
Mean and SEM

###ELBO
Compare ELBO of VAE and ED-VAE

In [ ]:
# AUC
df = pd.DataFrame((
    round(avg_auc_elbo_final_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_elbo_final_results[column,'auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_auc_elbo_final_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_elbo_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_auc_elbo_class_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_elbo_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_auc_elbo_class_results.keys().get_level_values(0)[::2]
    round(avg_auc_elbo_class_results[column,'auc_mean']*100,1).astype(str) for column in avg_auc_elbo_class_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_elbo_class_results.keys().get_level_values(0)[::2]).T)
df

,vae,ed-vae
0,99.7±0.0,99.8±0.0
1,99.9±0.0,99.9±0.0
2,93.0±0.4,95.3±0.4
3,94.6±0.4,95.8±0.3
4,95.0±0.2,96.3±0.3
5,95.8±0.2,96.7±0.2
6,98.9±0.1,99.3±0.1
7,96.5±0.2,97.3±0.2
8,89.5±0.4,91.1±0.5
9,97.6±0.1,98.3±0.1


In [ ]:
df.style.apply(highlight_max, axis=1)

,vae,ed-vae
0,99.7±0.0,99.8±0.0
1,99.9±0.0,99.9±0.0
2,93.0±0.4,95.3±0.4
3,94.6±0.4,95.8±0.3
4,95.0±0.2,96.3±0.3
5,95.8±0.2,96.7±0.2
6,98.9±0.1,99.3±0.1
7,96.5±0.2,97.3±0.2
8,89.5±0.4,91.1±0.5
9,97.6±0.1,98.3±0.1


In [ ]:
# PR-AUC
df = pd.DataFrame((
    round(avg_pr_elbo_final_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_elbo_final_results[column,'pr_auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_pr_elbo_final_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_elbo_final_results.keys().get_level_values(0)[::2]).T  
df = df.append(pd.DataFrame((
    #round(avg_pr_elbo_class_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_elbo_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_pr_elbo_class_results.keys().get_level_values(0)[::2]
    round(avg_pr_elbo_class_results[column,'pr_auc_mean']*100,1).astype(str) for column in avg_pr_elbo_class_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_elbo_class_results.keys().get_level_values(0)[::2]).T)
df

,vae,ed-vae
0,98.8±0.1,99.1±0.1
1,99.6±0.0,99.6±0.0
2,84.7±0.7,88.3±0.7
3,86.0±0.7,88.4±0.6
4,89.5±0.4,91.2±0.5
5,88.5±0.4,90.0±0.4
6,96.2±0.3,97.3±0.2
7,91.3±0.4,92.5±0.3
8,80.9±0.6,84.5±0.5
9,92.0±0.3,93.7±0.2


In [ ]:
df.style.apply(highlight_max, axis=1)

,vae,ed-vae
0,98.8±0.1,99.1±0.1
1,99.6±0.0,99.6±0.0
2,84.7±0.7,88.3±0.7
3,86.0±0.7,88.4±0.6
4,89.5±0.4,91.2±0.5
5,88.5±0.4,90.0±0.4
6,96.2±0.3,97.3±0.2
7,91.3±0.4,92.5±0.3
8,80.9±0.6,84.5±0.5
9,92.0±0.3,93.7±0.2


### $R_{error}$
Compare Reconstruction Error of VAE and ED-VAE

In [ ]:
# AUC
df = pd.DataFrame((
    round(avg_auc_rec_final_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_rec_final_results[column,'auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_auc_rec_final_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_rec_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_auc_rec_class_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_rec_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_auc_rec_class_results.keys().get_level_values(0)[::2]
    round(avg_auc_rec_class_results[column,'auc_mean']*100,1).astype(str) for column in avg_auc_rec_class_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_rec_class_results.keys().get_level_values(0)[::2]).T)
df

,vae,ed-vae
0,99.7±0.0,99.8±0.0
1,99.9±0.0,99.9±0.0
2,92.7±0.4,95.6±0.3
3,94.2±0.3,95.7±0.3
4,94.7±0.3,96.4±0.4
5,95.9±0.2,96.9±0.2
6,99.1±0.1,99.4±0.1
7,96.0±0.2,97.0±0.2
8,89.5±0.4,91.1±0.6
9,97.8±0.1,98.6±0.0


In [ ]:
df.style.apply(highlight_max, axis=1)

,vae,ed-vae
0,99.7±0.0,99.8±0.0
1,99.9±0.0,99.9±0.0
2,92.7±0.4,95.6±0.3
3,94.2±0.3,95.7±0.3
4,94.7±0.3,96.4±0.4
5,95.9±0.2,96.9±0.2
6,99.1±0.1,99.4±0.1
7,96.0±0.2,97.0±0.2
8,89.5±0.4,91.1±0.6
9,97.8±0.1,98.6±0.0


In [ ]:
# PR-AUC
df = pd.DataFrame((
    round(avg_pr_rec_final_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_rec_final_results[column,'pr_auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_pr_rec_final_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_rec_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_pr_rec_class_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_rec_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_pr_rec_class_results.keys().get_level_values(0)[::2]
    round(avg_pr_rec_class_results[column,'pr_auc_mean']*100,1).astype(str) for column in avg_pr_rec_class_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_rec_class_results.keys().get_level_values(0)[::2]).T)
df

,vae,ed-vae
0,98.8±0.1,99.1±0.1
1,99.7±0.0,99.6±0.0
2,84.2±0.7,88.6±0.7
3,85.8±0.6,88.5±0.5
4,89.8±0.5,91.7±0.6
5,89.4±0.3,91.0±0.4
6,96.7±0.3,97.7±0.2
7,90.9±0.3,92.1±0.3
8,81.8±0.6,85.6±0.5
9,92.9±0.2,94.6±0.1


In [ ]:
df.style.apply(highlight_max, axis=1)

,vae,ed-vae
0,98.8±0.1,99.1±0.1
1,99.7±0.0,99.6±0.0
2,84.2±0.7,88.6±0.7
3,85.8±0.6,88.5±0.5
4,89.8±0.5,91.7±0.6
5,89.4±0.3,91.0±0.4
6,96.7±0.3,97.7±0.2
7,90.9±0.3,92.1±0.3
8,81.8±0.6,85.6±0.5
9,92.9±0.2,94.6±0.1
